#### Author : Saurabh & Chee Weng
#### Objective : 
1. To enable users to generate output for books description in the style that users want. 
2. Being able to try Gemini, Gemini with Grounding, and Anthropic latest models too.
3. Although not shown but this approach can be extended to llama3.1 and other models. 

In [5]:
!pip install google-cloud-aiplatform


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}
! mkdir output

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

Updated property [core/project].
Creating gs://my-project-0004-346516-pytorch112kagglewbi-asia-southeast1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.
mkdir: cannot create directory ‘output’: File exists


### Generation just using Gemini (without grounding) 

In [7]:
import vertexai
from vertexai.preview.generative_models import GenerationConfig, GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel("gemini-1.5-pro-001")

def generate_call_to_action(title, author, description):
    prompt = f"""
    Given the following book information:
    Title: {title}
    Author: {author}
    Description: {description}

    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.
    """

    response = model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0.7,
            max_output_tokens=50,
        ),
    )

    return response.text

# Example usage
title = "Killer Croc hunter / written by Scott Sonneborn ; illustrated by Mike DeCarlo, Erik Doescher, and Lee Loughridge ; Batman created by Bob Kane."
author = "Scott Sonneborn"
description = "As a pro wrestler, Killer Croc bullied his opponents inside the ring. As a cutthroat criminal, this ferocious freak forces his competition out of town. Next on his hoodlum hit list is Kite Man, but the second-rate super-villain isn't leaving Gotham without a fight. He's partnering with Batman, the world's greatest detective, to wrangle up the reptilian rogue and take him down for the count"

call_to_action = generate_call_to_action(title, author, description)
print(call_to_action)

💥 **He's big, he's bad, he's Killer Croc!** 💥

Can Batman and the unlikely hero Kite Man stop this reptilian rogue before Gotham becomes his swamp? Dive into a world of bone-cr


### Generation just using Gemini with Grounding

In [8]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

# TODO (developer): update project_id
vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel("gemini-1.5-pro-001")

# Use Google Search for grounding
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

# prompt = "When is the next total solar eclipse in US?"

prompt = f"""
    Given the following book information:
    Title: {title}
    Author: {author}
    Description: {description}

    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.
    """
    
response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

print(response.text)

Can Batman stop Killer Croc's reign of terror? Killer Croc is targeting Gotham's villains, and only the Dark Knight can stop him! Will Batman and Kite Man team up to defeat this reptilian rogue? Find out in "Killer Croc Hunter"! 



### Use Clause model
##### Do enable clause model by filling up form from Model Garden first. 

In [9]:
from anthropic import AnthropicVertex

LOCATION="europe-west1" # or "us-east5"

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

message = client.messages.create(
  max_tokens=1024,
  messages=[
    {
      "role": "user",
      "content": prompt,
    }
  ],
  model="claude-3-5-sonnet@20240620",
)
print(message.model_dump_json(indent=2))

{
  "id": "msg_vrtx_01Dh3dwUbgVrsheYF8L7rsKU",
  "content": [
    {
      "text": "Attention Batman fans and action lovers! Dive into the thrilling world of \"Killer Croc Hunter\" by Scott Sonneborn. Watch as the Dark Knight teams up with an unlikely ally to take down Gotham's most ferocious reptilian villain! Will Batman and Kite Man's odd partnership be enough to stop Killer Croc's reign of terror? Find out in this action-packed adventure that'll keep you on the edge of your seat. Don't miss out – grab your copy today and join the hunt!",
      "type": "text"
    }
  ],
  "model": "claude-3-5-sonnet-20240620",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "input_tokens": 210,
    "output_tokens": 116
  }
}


### Scaling to read all the inputs from a CSV files and write back all the outputs to csv file


In [10]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('input.csv') 

# Display the first 5 rows
print(df.head())

                                               Title           Author  \
0  Killer Croc hunter / written by Scott Sonnebor...  Scott Sonneborn   

                                         Description  \
0  As a pro wrestler, Killer Croc bullied his opp...   

                         AI-generated-Call-to-Action  
0  Get ready for an epic showdown! Join Batman an...  


In [19]:
import csv
import vertexai
from vertexai.preview.generative_models import GenerationConfig, GenerativeModel

vertexai.init(project=PROJECT_ID, location="us-central1")
model = GenerativeModel("gemini-1.5-pro-001")


def generate_call_to_action(title, author, description):
    prompt = f"""
    Given the following book information:
    Title: {title}
    Author: {author}
    Description: {description}

    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.
    """

    response = model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0.7,
            max_output_tokens=50,
        ),
    )

    return response.text


tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

def generate_call_to_action_withGrounding(title, author, description):
    prompt = f"""    Given the following book information:    Title: {title}    Author: {author}    Description: {description}    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.    """

    response = model.generate_content(
        prompt,
        tools=[tool],  # Include the grounding tool
        generation_config=GenerationConfig(
            temperature=0.7,  # You can adjust this for creativity
            max_output_tokens=50,
        ),
    )

    return response.text


from anthropic import AnthropicVertex

def generate_response_with_anthropic(title, author, description, location="europe-west1", model_name="claude-3-5-sonnet@20240620", max_tokens=1024):
    """
    Generates a response using the Anthropic Claude model.

    Args:
    prompt: The prompt text for the model.
    location: The cloud region where the model is deployed.
    model_name: The specific Anthropic model to use.
    max_tokens: The maximum number of tokens in the generated response.

    Returns:
    The generated text response.
    """

    prompt = f"""
        Given the following book information:
        Title: {title}
        Author: {author}
        Description: {description}

        Generate an engaging call-to-action text to promote this book. 
        The text should be concise, exciting, and encourage readers to check out the book.
    """
    client = AnthropicVertex(region=location, project_id=PROJECT_ID)  # Assuming PROJECT_ID is defined elsewhere

    message = client.messages.create(
      max_tokens=max_tokens,
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model=model_name,
    )

    return message.content


# Read CSV and process each row
with open("input.csv", "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row (if present)

    for row in reader:
        title, author, description, old_call_to_action = row  # Assuming columns are in this order
        call_to_action_withgemini = generate_call_to_action(title, author, description)
        call_to_action_withgrounding = generate_call_to_action_withGrounding(title, author, description)
        call_to_action_anthropic = generate_response_with_anthropic(title, author, description)
        print(f"Call to action for '{title}': {call_to_action_withgemini}")

Call to action for 'Killer Croc hunter / written by Scott Sonneborn ; illustrated by Mike DeCarlo, Erik Doescher, and Lee Loughridge ; Batman created by Bob Kane.': **Option 1 (Short & Punchy):**

Killer Croc is on a rampage! Can Batman and Kite Man stop this reptilian rogue before it's too late? Find out in KILLER CROC HUNTER! 

**Option


In [20]:
# call_to_action = generate_response_with_anthropic(title, author, description)

In [21]:
# print(call_to_action)

#### Write the outputs back to csv file

In [22]:
import csv

# Read CSV and process each row
with open("input.csv", "r") as file:
    reader = csv.reader(file)
    header = next(reader)  # Store the header row
    rows = list(reader)  # Read all remaining rows into a list

# Generate call-to-actions and update rows
for row in rows:
    title, author, description, old_call_to_action = row
    call_to_action_withgemini = generate_call_to_action(title, author, description)
    call_to_action_withgrounding = generate_call_to_action_withGrounding(title, author, description)
    call_to_action_anthropic = generate_response_with_anthropic(title, author, description)
    
    row.extend([call_to_action_withgemini, call_to_action_withgrounding, call_to_action_anthropic])

# Write the updated data back to the CSV
with open("output.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header + ["Call to Action with Gemini", "Call to Action with Grounding", "Call to Action with Anthropic"])
    writer.writerows(rows)

# (Optional) If you want to write these to CSS directly:
with open("output.css", "w") as css_file:
    for row in rows:
        title, author, description, old_call_to_action, call_to_action_withgemini, call_to_action_withgrounding, call_to_action_anthropic = row
        # You'll need to define how you want to structure the CSS
        # Here's a basic example assuming you have a class for each title:
        css_file.write(f".{title} {{\n")
        css_file.write(f"  /* Other CSS properties */\n")
        css_file.write(f"  --call-to-action-withgemini: '{call_to_action_withgemini}';\n")
        css_file.write(f"  --call-to-action-withgrounding: '{call_to_action_withgrounding}';\n")
        css_file.write(f"  --call-to-action-withanthropic: '{call_to_action_anthropic}';\n")
        css_file.write("}\n\n")

## Impoving using a few shot examples 



In [13]:
few_shot_examples =  """
**Example 1**

[INPUT] Title
The Martian

[INPUT] Author
Andy Weir

[INPUT] Description
An astronaut becomes stranded on Mars after his team assumes him dead, and must rely on his ingenuity to find a way to signal to Earth that he is alive.

[OUTPUT] AI-generated Call-to-Action
Stranded on Mars, one man's fight for survival begins. Can he defy the odds and make it home? Experience the thrilling adventure of *The Martian* today! 

**Example 2**

[INPUT] Title
Pride and Prejudice

[INPUT] Author
Jane Austen

[INPUT] Description
Follow the spirited Elizabeth Bennet as she navigates the complexities of love and societal expectations in 19th-century England. Will she find happiness amidst the prejudices and pride of her world?

[OUTPUT] AI-generated Call-to-Action
Wit, romance, and social satire intertwine in this timeless classic. Discover the enduring charm of *Pride and Prejudice* - a must-read for every generation!

**Example 3**

[INPUT] Title
The Hunger Games

[INPUT] Author
Suzanne Collins

[INPUT] Author
In a dystopian future, Katniss Everdeen volunteers to take her younger sister's place in the Hunger Games, a televised event in which teenagers must fight to the death.

[OUTPUT] AI-generated Call-to-Action
Enter the arena! Join Katniss in a fight for survival and rebellion against a tyrannical regime. *The Hunger Games* - an unforgettable story of courage and defiance.


"""

In [23]:
prompt_few_shots = f"""
    **Few-Shot Examples for Generating Book Call-to-Actions**

{few_shot_examples}

    **Book Information**

    Title: {title}
    Author: {author}
    Description: {description}

    **Task**

    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.
"""

In [24]:
# generate_call_to_action_withGrounding_fewshot

def generate_call_to_action_fewshot(title, author, description):
    prompt = f"""
    Given the following book information:
    Title: {title}
    Author: {author}
    Description: {description}

    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.
    """

    response = model.generate_content(
        prompt_few_shots,
        generation_config=GenerationConfig(
            temperature=0.7,
            max_output_tokens=50,
        ),
    )

    return response.text


tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

def generate_call_to_action_withGrounding_fewshot(title, author, description):
    prompt = f"""    Given the following book information:    Title: {title}    Author: {author}    Description: {description}    Generate an engaging call-to-action text to promote this book. The text should be concise, exciting, and encourage readers to check out the book.    """

    response = model.generate_content(
        prompt_few_shots,
        tools=[tool],  # Include the grounding tool
        generation_config=GenerationConfig(
            temperature=0.7,  # You can adjust this for creativity
            max_output_tokens=50,
        ),
    )

    return response.text


def generate_response_with_anthropic_fewshot(title, author, description, location="europe-west1", model_name="claude-3-5-sonnet@20240620", max_tokens=1024):
    client = AnthropicVertex(region=location, project_id=PROJECT_ID)  # Assuming PROJECT_ID is defined elsewhere

    message = client.messages.create(
      max_tokens=max_tokens,
      messages=[
          {
              "role": "user",
              "content": prompt_few_shots,
          }
      ],
      model=model_name,
    )

    return message.content


# Read CSV and process each row
with open("input.csv", "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row (if present)

    for row in reader:
        title, author, description, old_call_to_action = row  # Assuming columns are in this order
        call_to_action = generate_call_to_action_withGrounding_fewshot(title, author, description)
        print(f"Call to action for '{title}': {call_to_action}")

Call to action for 'Killer Croc hunter / written by Scott Sonneborn ; illustrated by Mike DeCarlo, Erik Doescher, and Lee Loughridge ; Batman created by Bob Kane.': Can Batman stop Killer Croc from terrorizing Gotham City? When Killer Croc targets Kite Man, the unlikely duo of Batman and Kite Man must team up to stop him. Find out if they can defeat the reptilian rogue in *Killer Croc


#### Same thing runn the above code via csv files here 


In [26]:
import csv

# Read CSV and process each row
with open("input.csv", "r") as file:
    reader = csv.reader(file)
    header = next(reader)  # Store the header row
    rows = list(reader)  # Read all remaining rows into a list

# Generate call-to-actions and update rows
for row in rows:
    title, author, description, old_call_to_action = row
    call_to_action_withgemini_fewshot = generate_call_to_action_fewshot(title, author, description)
    call_to_action_withgrounding_fewshot = generate_call_to_action_withGrounding_fewshot(title, author, description)
    call_to_action_anthropic_fewshot = generate_response_with_anthropic_fewshot(title, author, description)
    
    row.extend([call_to_action_withgemini, call_to_action_withgrounding, call_to_action_anthropic])

# Write the updated data back to the CSV
with open("output_fewshot.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header + ["Call to Action with Gemini_fewshot", "Call to Action with Grounding_fewshot", "Call to Action with Anthropic_fewshot"])
    writer.writerows(rows)

# (Optional) If you want to write these to CSS directly:
with open("output_fewshot.css", "w") as css_file:
    for row in rows:
        title, author, description, old_call_to_action, call_to_action_withgemini_fewshot, call_to_action_withgrounding_fewshot, call_to_action_anthropic_fewshot = row
        # You'll need to define how you want to structure the CSS
        # Here's a basic example assuming you have a class for each title:
        css_file.write(f".{title} {{\n")
        css_file.write(f"  /* Other CSS properties */\n")
        css_file.write(f"  --call-to-action-withgemini: '{call_to_action_withgemini_fewshot}';\n")
        css_file.write(f"  --call-to-action-withgrounding: '{call_to_action_withgrounding_fewshot}';\n")
        css_file.write(f"  --call-to-action-withanthropic: '{call_to_action_anthropic_fewshot}';\n")
        css_file.write("}\n\n")